In [1]:
id: str = 'rise'
spektrix_event_ids: str = '4801ANHPDLLRCQPPBDBQTRQJMPNKSLKPJ'
venue_postcode: str = 'BD1 1LA'
rosterfy_event_ids: str = '25890'
project_ids: str = 'recf4tPyjoXi81NDo'

dates = ['2025-01-10 18:30:00', '2025-01-11 18:30:00']

In [2]:
import json
from datetime import datetime

from utils.paths import SITE
from utils.process.event import Programme, Sustainability, Tickets, Volunteers

In [3]:
OUTPUT_DIR = SITE / 'events' / id / '_data/event'
PROGRAMME_DIR = OUTPUT_DIR / 'programme'
PROGRAMME_DIR.mkdir(exist_ok=True, parents=True)
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)
VOLUNTEERS_DIR = OUTPUT_DIR / 'volunteers'
SUSTAINABILITY_DIR = OUTPUT_DIR / 'sustainability'

In [4]:
event_times = [datetime.fromisoformat(d) for d in dates]

In [5]:
if project_ids:
    programme = Programme(project_ids.split())
    with open(PROGRAMME_DIR / 'summary.json', 'w') as f:
        json.dump(programme.summarise(), f, indent=2)

In [6]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')
    tickets.types().tocsv(TICKETS_DIR / 'by_type.csv')

In [7]:
if rosterfy_event_ids:
    VOLUNTEERS_DIR.mkdir(exist_ok=True, parents=True)
    volunteers = Volunteers(str(rosterfy_event_ids).split())
    volunteers.summarise().tocsv(VOLUNTEERS_DIR / 'summary.csv')

In [8]:
if project_ids:
    SUSTAINABILITY_DIR.mkdir(exist_ok=True, parents=True)
    sustainability = Sustainability(project_ids.split()).summarise()
    sustainability.tocsv(SUSTAINABILITY_DIR / 'breakdown.csv')
    sustainability.select(lambda r: r.impact_type == 'ALL' and r.scope == 'ALL').cut('tCO2e').tojson(SUSTAINABILITY_DIR / 'summary.json', lines=True)